In [1]:
import deeplake as dl
# ds = dl.load('/mnt/teton/vpt/data/benchmark_datasets/TVQA/_deeplake/whisper_results_bbt_audios')
ds = dl.load('/mnt/teton/vpt/data/deeplake_handpicked/CLIP_encode_results_handpicked_downloads/')
ds.summary()

/mnt/teton/vpt/data/deeplake_handpicked/CLIP_encode_results_handpicked_downloads/ loaded successfully.
Dataset(path='/mnt/teton/vpt/data/deeplake_handpicked/CLIP_encode_results_handpicked_downloads/', tensors=['caption', 'caption_embedding', 'clip_last_hidden_states', 'clip_pooled_embedding', 'frames', 'segment_metadata', 'timestamp', 'video_filename', 'video_filepath'])

         tensor            htype                shape                dtype  compression
         -------          -------              -------              -------  ------- 
         caption           text              (266689, 1)              str     None   
    caption_embedding     generic      (266689, 16:122, 1024)       float16   None   
 clip_last_hidden_states  generic        (266689, 577, 1024)        float32    lz4   
  clip_pooled_embedding   generic          (266689, 1024)           float32    lz4   
         frames            image   (266689, 240:1080, 204:1920, 3)   uint8    jpeg   
    segment_metadata 

In [7]:
# ds.commit('Great working handpicked dataset. Feb-24-2023')

ds[0:48].save_view(id='eval')

'/mnt/teton/vpt/data/deeplake_handpicked/CLIP_encode_results_handpicked_downloads/.queries/eval'

In [4]:
eval.summary()

eval.copy('/mnt/teton/vpt/data/deeplake_handpicked/CLIP_encode_results_handpicked_downloads_eval')

Dataset(path='/mnt/teton/vpt/data/deeplake_handpicked/CLIP_encode_results_handpicked_downloads/', index=Index([slice(0, 48, None)]), tensors=['caption', 'caption_embedding', 'clip_last_hidden_states', 'clip_pooled_embedding', 'frames', 'segment_metadata', 'timestamp', 'video_filename', 'video_filepath'])

         tensor            htype                shape                dtype  compression
         -------          -------              -------              -------  ------- 
         caption           text                (48, 1)                str     None   
    caption_embedding     generic      (266689, 16:122, 1024)       float16   None   
 clip_last_hidden_states  generic          (48, 577, 1024)          float32    lz4   
  clip_pooled_embedding   generic            (48, 1024)             float32    lz4   
         frames            image   (266689, 240:1080, 204:1920, 3)   uint8    jpeg   
    segment_metadata       text                (48, 1)                str     None   
   

AttributeError: '<class 'deeplake.core.dataset.dataset.Dataset'>' object has no attribute 'save'

In [7]:
for i, val in enumerate(ds):
  assert val['caption'].numpy() is not None, f'Caption is None at index {i}'
  print(i, val['caption'].numpy())

0 ['THANK YOU VERY MUCH THANKS THAT IS UNBELIEVABLE GOOD FOR BURT DAMN THE MACARTHUR GENIUS']
1 ["GRANT COULDN'T HAVE HAPPENED TO A NICER GUY EVERYTHING IS STUPID AND I WANT TO"]
2 ["GO HOME THAT'S SHELDON'S WAY OF SAYING HE'S PROUD OF BURT TOO YOU KNOW HE'S"]
3 ["NOT EVEN CALLED THE GENIUS GRANT IT'S THE MACARTHUR FELLOWSHIP JUST LIKE IT'S NOT FRANKENSTEIN"]
4 ["IT'S FRANKENSTEIN'S MONSTER WHICH BRINGS US BACK TO THAT HULKING SIMPLETON OVER THERE AND IF"]
5 ["ANYONE'S A GENIUS IT'S ME THE WAY I BROUGHT THAT FULL CIRCLE HOW MUCH MONEY"]
6 ["DID HE WIN OVER HALF A MILLION DOLLARS WOW AND HE DOESN'T HAVE TO USE"]
7 ['IT FOR RESEARCH HE CAN DO WHATEVER HE WANTS WITH IT OH GOOD MAYBE HE']
8 ['CAN BUILD A NICER BRIDGE TO LIVE UNDER HERE HE COMES WAIT BE POLITE I']
9 ['KNOW HOW TO BEHAVE DO YOU HEY CONGRATULATIONS BURT YEAH GOOD FOR YOU THANKS I']
10 ["WAS AS SURPRISED AS ANYBODY WHEN THEY CALLED AND TOLD ME I WON I DIDN'T"]
11 ['BELIEVE THEM AND THEN THEY SAID NO YOU REALLY WON AND THEN I S

In [11]:
# ds[18322]['caption'].numpy()
ds[18320]['caption'].numpy()

array(["AFTER ALL AND YOU YOU SAID YOU WEREN'T COMING HERE EITHER I WOULD TAKE ACCIDENT"],
      dtype='<U79')

In [ ]:
from transformers import (AutoModelWithLMHead, T5Config, T5ForConditionalGeneration, T5Model, T5Tokenizer)
# 783,150,080. ~800M params.
t5 = T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-large",
                                                torch_dtype=torch.float32,
                                                low_cpu_mem_usage=False).to(device)  # float16, True
t5_tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-large", return_special_tokens_mask=True)
learning_rate = 1e-4  # also good: 3e-4
# low_cpu_mem_usage(bool, optional) — Tries to not use more than 1x model size in CPU memory (including peak memory) while loading the model. experimental.
optimizer = torch.optim.AdamW(params=t5.parameters(),
                              lr=learning_rate)  # Typically, 1e-4 and 3e-4 work well for most problems
pytorch_total_params = sum(p.numel() for p in t5.parameters())
pytorch_total_params

In [1]:
from transformers import (AutoModelWithLMHead, T5Config, T5ForConditionalGeneration, T5Model, T5Tokenizer)
T5Config("google/t5-v1_1-xl")

/home/kastan/utils/mambaforge/envs/nlp_v2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


T5Config {
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.25.0.dev0",
  "use_cache": true,
  "vocab_size": "google/t5-v1_1-xl"
}

In [2]:
T5Config("google/t5-v1_1-xxl")

T5Config {
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.25.0.dev0",
  "use_cache": true,
  "vocab_size": "google/t5-v1_1-xxl"
}

In [ ]:
import jsonlines
import os
import json
import glob
import numpy as np
import pathlib
from numpy import load
import torch
import lovely_tensors as lt
lt.monkey_patch()
import tqdm
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

# DEBUGGING 
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
dir_name = "parallel_15"
REMOTE_WHISPER_FILE = f'/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/{dir_name}_whisper_output.jsonl'
REMOTE_CLIP_DIR  = f'/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/{dir_name}_clip_output'
REMOTE_SCENE_FILE = f'/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/{dir_name}_scene_output.jsonl'

In [ ]:
# Instantiate clip
import clip

MODEL_SIZE = 'ViT-L/14@336px'  # Best models are (1st) ViT-L/14@336px and (2nd) ViT-L/14. I don't recommend going lower.  
clip_instance, clip_preprocess = clip.load(MODEL_SIZE, device)

In [ ]:
from transformers import GPT2Tokenizer, OPTForCausalLM
t5 = OPTForCausalLM.from_pretrained("facebook/opt-125m").to(device)
t5_tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-125m")

learning_rate       = 1e-4  # also good: 3e-4
optimizer = torch.optim.AdamW(params =  t5.parameters(), lr=learning_rate) # Typically, 1e-4 and 3e-4 work well for most problems

# prompt = "Hey, are you consciours? Can you talk to me?"
# inputs = tokenizer(prompt, return_tensors="pt")

# # Generate
# generate_ids = model.generate(inputs.input_ids, max_length=30)
# tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
from transformers import RobertaForCausalLM, RobertaTokenizer
MODEL_NAME = "roberta-base"
t5 = RobertaForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float32, low_cpu_mem_usage=False).to(device) # float16, True
t5_tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME, return_special_tokens_mask=True)

In [ ]:
# T5
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Model, T5Config, AutoModelWithLMHead

'''
MODEL SELECTION

T5 V1.1 --  https://huggingface.co/docs/transformers/model_doc/t5v1.1 && https://github.com/google-research/text-to-text-transfer-transformer/blob/main/released_checkpoints.md#t511
small - base - large - 3b/xl - 11b/xxl

OG: t5-small

'google/t5-base-lm-adapt' # largest on my server (without float16)
'google/t5-xl-lm-adapt'

google/t5-v1_1-large
'''

# MODEL_SIZE = "t5-base"
MODEL_NAME = "google/t5-v1_1-base"
# MODEL_NAME = "google/t5-base-lm-adapt"
# config = T5Config.from_pretrained(MODEL_NAME)
t5 = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, torch_dtype=torch.float32, low_cpu_mem_usage=False).to(device) # float16, True
t5_tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, return_special_tokens_mask=True)
# low_cpu_mem_usage(bool, optional) — Tries to not use more than 1x model size in CPU memory (including peak memory) while loading the model. experimental.
learning_rate       = 1e-4  # also good: 3e-4
optimizer = torch.optim.AdamW(params =  t5.parameters(), lr=learning_rate) # Typically, 1e-4 and 3e-4 work well for most problems


In [ ]:
type(t5.lm_head)

In [ ]:
# Cockpit ML Debugger
# https://cockpit.readthedocs.io/en/latest/examples/01_basic_fmnist.html
from backpack import extend
from cockpit import Cockpit, CockpitPlotter
from cockpit.utils.configuration import configuration

t5 = extend(t5)

individual_loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Create Cockpit and a plotter
cockpit = Cockpit(t5.parameters(), quantities=configuration("full"))
plotter = CockpitPlotter()

In [ ]:
''' just for trying the debugger cockpit'''


# Iterate through the batch
clip_15 = os.listdir(REMOTE_CLIP_DIR)

# Initialize embeddings
one_input_shape = [1, 768, 768]
att_mask_shape = [1, 768]
embed_shape = [1, 768]

input_embeds_arr = torch.zeros(one_input_shape).to(device) # .astype(np.float16)
attn_mask_arr    = torch.zeros(att_mask_shape).to(device)
attn_mask_arr[0][0] = 1
attn_mask_arr[0][1] = 1
attn_mask_arr[0][2] = 1 # no clip

t5.train()

global_step = 0 

with jsonlines.open(REMOTE_SCENE_FILE, 'r') as scene_reader:
    # Zipping the scene graph with the clip + whisper embeddings
    
    # itr over videos
    for scene_seg_list, clip_npz_path in tqdm.tqdm(zip(scene_reader, glob.glob(os.path.join(REMOTE_CLIP_DIR, '*'), recursive = True))):
        try:
            np_loaded = np.load(clip_npz_path, allow_pickle=True)
        except Exception as e:
            print(f"Failed to load compressed numpy: {e}")
            continue
        object_list_of_str = []
        scene_seg_list = json.loads(scene_seg_list)
        
        # iterate over segments
        for segment_index in range(np_loaded['arr_0'].item()['total_segments']):
            # print(np_loaded[f'arr_{segment_index}'].item()['captions'])
            frame_embedding       = np_loaded[f'arr_{segment_index}'].item()['frame_embeddings']
            caption_embedding     = np_loaded[f'arr_{segment_index}'].item()['text_caption_embeddings']
            whisper_text_captions = np_loaded[f'arr_{segment_index}'].item()['captions']
            
            frame_embedding       = torch.from_numpy(frame_embedding.reshape((768,))).to(device)
            caption_embedding     = torch.from_numpy(caption_embedding).to(device)

            # Update embedding array
            input_embeds_arr[0][0] = frame_embedding
            input_embeds_arr[0][1] = caption_embedding
            
            # print("Input shapes:")
            # print(caption_embedding)
            print(frame_embedding)
            labels = t5_tokenizer(whisper_text_captions, return_tensors="pt").input_ids.to(device)
            # outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, decoder_inputs_embeds=input_embeds_arr)
            outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, labels=labels, return_dict=True)
            # outputs = t5.forward(inputs_embeds=input_embeds_arr, labels=labels)
            loss = outputs[0]
            print("loss")
            print(loss)
            logits = outputs[1]
            # print("logits")
            # print(logits)
            individual_losses = individual_loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            # print("individual_losses")
            # print(individual_losses)
            # loss = 
            for item in outputs:
                print(item)
            ''' backwards pass '''
            # loss.sum().backward()
            # backward pass
            with cockpit(
                global_step,
                info={
                    "batch_size": one_input_shape[0],
                    "individual_losses": individual_losses,
                    "loss": loss,
                    "optimizer": optimizer,
                },
            ):
                loss.backward(create_graph=cockpit.create_graph(global_step))
            
            # optimizer step
            optimizer.zero_grad()
            optimizer.step()
            global_step += 1
            print(f"step: {global_step}")
            plotter.plot(cockpit)
            
            print("Loss 👇👇👇")
            print(loss)
            break
        break


In [ ]:
# from transformers import RobertaConfig
# OPTConfig.from_pretrained("roberta-base")
from transformers import OPTConfig
OPTConfig.from_pretrained("facebook/opt-125m")

In [ ]:
''' More complete '''

# Initialize embeddings
# one_input_shape = [1, 768, 768]
# att_mask_shape = [1, 768]
# embed_shape = [1, 768]
one_input_shape = [1, 2048, 768]  # torch.FloatTensor of shape (batch_size, sequence_length, hidden_size)
att_mask_shape = [1, 2048]        # 
embed_shape = [1, 768]

input_embeds_arr = torch.zeros(one_input_shape, device=device)
attn_mask_arr    = torch.zeros(att_mask_shape, device=device)
labels    = torch.zeros(att_mask_shape, dtype=int, device=device)
print(labels.dtype)
labels[:len(labels)] = -100
attn_mask_arr[0][0] = 1
attn_mask_arr[0][1] = 1
# attn_mask_arr[0][2] = 1 # no clip

t5.train()

global_step = 0 

with jsonlines.open(REMOTE_SCENE_FILE, 'r') as scene_reader:
    # Zipping the scene graph with the clip + whisper embeddings
    # itr over videos
    for scene_seg_list, clip_npz_path in tqdm.tqdm(zip(scene_reader, glob.glob(os.path.join(REMOTE_CLIP_DIR, '*'), recursive = True))):
        try:
            np_loaded = np.load(clip_npz_path, allow_pickle=True)
        except Exception as e:
            print(f"Failed to load compressed numpy: {e}")
            continue
        object_list_of_str = []
        scene_seg_list = json.loads(scene_seg_list)
        
        # iterate over segments
        for segment_index in range(np_loaded['arr_0'].item()['total_segments']):
            frame_embedding       = np_loaded[f'arr_{segment_index}'].item()['frame_embeddings']
            caption_embedding     = np_loaded[f'arr_{segment_index}'].item()['text_caption_embeddings']
            whisper_text_captions = np_loaded[f'arr_{segment_index}'].item()['captions']
            
            frame_embedding       = torch.from_numpy(frame_embedding.reshape((768,))).to(device)
            caption_embedding     = torch.from_numpy(caption_embedding).to(device)

            scene_caption = scene_seg_list[segment_index]
            scene_caption = clip.tokenize(scene_caption).to(device)
            with torch.inference_mode(): # even faster than no_grad()
                scene_embedding = clip_instance.encode_text(scene_caption)
            scene_embedding = scene_embedding.reshape((768,))

            # Update embedding array
            input_embeds_arr[0][0] = frame_embedding
            input_embeds_arr[0][1] = caption_embedding
            input_embeds_arr[0][2] = scene_embedding
            
            print("Input shapes:")
            print(input_embeds_arr.shape)
            print(scene_embedding.shape)
            print(caption_embedding.shape)
            print(frame_embedding.shape)
            tokenized_labels = t5_tokenizer(whisper_text_captions, return_tensors="pt").input_ids.to(device)
            print("tokenized_labels")
            print(tokenized_labels)
            print("labels")
            print(labels)
            print(labels.dtype)
            labels[0][0:len(tokenized_labels[0])] = tokenized_labels[0]
            # outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, decoder_inputs_embeds=input_embeds_arr)
            # outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, labels=labels, return_dict=True)
            # outputs = t5.forward(inputs_embeds=input_embeds_arr, labels=labels)
            outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, labels=labels, return_dict=True, return_last_hidden_state=True)
            print(outputs['last_hidden_state'].shape)
            
            results = t5.lm_head(outputs['last_hidden_state'])
            print(results)
            # loss = outputs[0]
            # ''' backwards pass '''
            # loss.sum().backward()
            # # optimizer step
            # optimizer.zero_grad()
            # optimizer.step()
            # global_step += 1
            print(f"step: {global_step}")
            
            print("Loss 👇👇👇")
            print(loss)
            break
        break


In [ ]:
''' More complete -- NO CLIP (for convienence) '''

# Initialize embeddings
one_input_shape = [1, 768, 768]
att_mask_shape = [1, 768]
embed_shape = [1, 768]
# one_input_shape = [1, 2048, 768]  # torch.FloatTensor of shape (batch_size, sequence_length, hidden_size)
# att_mask_shape = [1, 2048]        # 
# embed_shape = [1, 768]

input_embeds_arr = torch.zeros(one_input_shape, device=device)
attn_mask_arr    = torch.zeros(att_mask_shape, device=device)
labels    = torch.zeros(att_mask_shape, dtype=int, device=device)
print(labels.dtype)
labels[:len(labels)] = -100
attn_mask_arr[0][0] = 1
attn_mask_arr[0][1] = 1
# attn_mask_arr[0][2] = 1 # no clip

t5.train()

global_step = 0 

with jsonlines.open(REMOTE_SCENE_FILE, 'r') as scene_reader:
    # Zipping the scene graph with the clip + whisper embeddings
    # itr over videos
    for scene_seg_list, clip_npz_path in tqdm.tqdm(zip(scene_reader, glob.glob(os.path.join(REMOTE_CLIP_DIR, '*'), recursive = True))):
        try:
            np_loaded = np.load(clip_npz_path, allow_pickle=True)
        except Exception as e:
            print(f"Failed to load compressed numpy: {e}")
            continue
        object_list_of_str = []
        scene_seg_list = json.loads(scene_seg_list)
        
        # iterate over segments
        for segment_index in range(np_loaded['arr_0'].item()['total_segments']):
            frame_embedding       = np_loaded[f'arr_{segment_index}'].item()['frame_embeddings']
            caption_embedding     = np_loaded[f'arr_{segment_index}'].item()['text_caption_embeddings']
            whisper_text_captions = np_loaded[f'arr_{segment_index}'].item()['captions']
            
            frame_embedding       = torch.from_numpy(frame_embedding.reshape((768,))).to(device)
            caption_embedding     = torch.from_numpy(caption_embedding).to(device)

            scene_caption = scene_seg_list[segment_index]
            # scene_caption = clip.tokenize(scene_caption).to(device)
            # with torch.inference_mode(): # even faster than no_grad()
                # scene_embedding = clip_instance.encode_text(scene_caption)
            # scene_embedding = scene_embedding.reshape((768,))

            # Update embedding array
            input_embeds_arr[0][0] = frame_embedding
            input_embeds_arr[0][1] = caption_embedding
            # input_embeds_arr[0][2] = scene_embedding
            
            print("Input shapes:")
            print(input_embeds_arr.shape)
            # print(scene_embedding.shape)
            print(caption_embedding.shape)
            print(frame_embedding.shape)
            tokenized_labels = t5_tokenizer(whisper_text_captions, return_tensors="pt").input_ids.to(device)
            print("tokenized_labels")
            print(tokenized_labels)
            print("labels")
            print(labels)
            print(labels.dtype)
            labels[0][0:len(tokenized_labels[0])] = tokenized_labels[0]
            # outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, decoder_inputs_embeds=input_embeds_arr)
            # outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, labels=labels, return_dict=True)
            # outputs = t5.forward(inputs_embeds=input_embeds_arr, labels=labels)
            
            decoder_input_ids = t5.forward()
            output_ids = t5.generate(attention_mask=attn_mask_arr, decoder_input_ids=decoder_input_ids, inputs_embeds=input_embeds_arr, max_length=100, num_beams=4)

            outputs = t5.forward(inputs_embeds=input_embeds_arr, attention_mask=attn_mask_arr, labels=labels, return_dict=True, output_hidden_states=True)
            print(outputs['last_hidden_state'].shape)
            
            results = t5.lm_head(outputs['last_hidden_state'])
            print(results)
            # loss = outputs[0]
            # ''' backwards pass '''
            # loss.sum().backward()
            # # optimizer step
            # optimizer.zero_grad()
            # optimizer.step()
            # global_step += 1
            print(f"step: {global_step}")
            
            print("Loss 👇👇👇")
            print(loss)
            break
        break


In [ ]:
tokenized_labels

In [ ]:
res = t5.lm_head(outputs['encoder_last_hidden_state'][0][0], shared_embedding=t5.shared)
res.plt()

In [ ]:
t5.get_output_layer_with_bias()

In [ ]:
outputs['encoder_last_hidden_state']

In [ ]:
decoder_input_ids = outputs['encoder_last_hidden_state']
output_ids = t5.generate(attention_mask=attn_mask_arr, inputs_embeds=input_embeds_arr, max_length=768, num_beams=4)

In [ ]:
print(t5_tokenizer.decode(output_ids[0], skip_special_tokens=True))

In [ ]:
outputs['encoder_last_hidden_state']

for i in range(40):
  print(i, "  ",outputs['encoder_last_hidden_state'][0][i])
  # display(outputs['encoder_last_hidden_state'][0][i].plt)

In [ ]:
# t5.save_pretrained("BIG_PENIS_PREVAILS")

In [ ]:
labels[0][0:27]
# len(tokenized_labels)

# Trying Inference with custom model

import jsonlines
import os
import json
import glob
import numpy as np
import pathlib
from numpy import load
import lovely_tensors as lt
lt.monkey_patch()
import tqdm

import jsonlines
import os
import json
import glob
import numpy as np
import pathlib
from numpy import load
import lovely_tensors as lt
lt.monkey_patch()
import tqdm

In [ ]:
import jsonlines
import os
import json
import glob
import numpy as np
import pathlib
from numpy import load
import torch
import lovely_tensors as lt
lt.monkey_patch()
import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

In [ ]:
# LOAD FINE-TUNED T5
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Model, T5Config
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import OPTForCausalLM

# MODEL_VERSION_NAME   = "opt_yt_pretrain_adamW_1e-3_iter69000"
# TOKENIZER_NAME       = "facebook/opt-125m"

# BASE_DIR            = '/scratch/bbki/kastanday/whisper'
# BASE_DIR            = '/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train'
# MODEL_SAVE_PATH     = f'{BASE_DIR}/MODEL_CHECKPOINTS/{MODEL_VERSION_NAME}'

TOKENIZER_NAME       = "google/t5-v1_1-base"
t5     = AutoModelWithLMHead.from_pretrained("/mnt/storage_hdd/thesis/MODEL_CHECKPOINTS/T5_labels_are_half_batch_15_adamW_iter622500").to(device)
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

In [ ]:
# Instantiate clip
import clip

MODEL_SIZE = 'ViT-L/14@336px'  # Best models are (1st) ViT-L/14@336px and (2nd) ViT-L/14. I don't recommend going lower.  
clip_instance, clip_preprocess = clip.load(MODEL_SIZE, device)

In [ ]:
# get standard image and embed with CLIP.
import deeplake as dl
clip_dataset_path = f'/mnt/storage_ssd/dummy_clip_results_parallel_15'
ds = dl.load(clip_dataset_path)
print(ds.summary())

frame_embedding = ds.pooled_clip_embedding[0].numpy()
frame_embedding.shape

In [ ]:
# PROMPT = "Hey guys welcome back to the channel, "
# PROMPT = "I REALLY LIKE EATING ASS"
# PROMPT = "TODAY WE ARE GOING TO"
PROMPT = "IN THIS VIDEO WE WILL"
# PROMPT = "THE MOST COMMON WORD IN THE ENGLISH LANGUAGE"
# PROMPT = "IN THIS VIDEO WE WILL"
# PROMPT = "I LOVE VIDEO GAMES BECAUSE I LOVE KILLING"
PROMPT = "The secret to cooking a good burrito is"


# OLD/naive inference
# input_ids = tokenizer(PROMPT, return_tensors="pt").input_ids
# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0]))


# Initialize embeddings
one_input_shape = [1, 768, 768]
att_mask_shape = [1, 768]
input_embeds_arr = torch.zeros(one_input_shape, device=device)
attn_mask_arr    = torch.zeros(att_mask_shape, device=device)
# labels    = torch.zeros(att_mask_shape, dtype=int, device=device)
# labels[:] = -100

# device = 'cpu'
caption_tokenized = clip.tokenize(PROMPT).to(device)
with torch.inference_mode():
    caption_embedding = clip_instance.encode_text(caption_tokenized)
caption_embedding = caption_embedding.reshape((768,))

attn_mask_arr[0][0] = 1
attn_mask_arr[0][1] = 1
attn_mask_arr[0][2] = 1
input_embeds_arr[0][0] = torch.tensor(frame_embedding)
input_embeds_arr[0][1] = caption_embedding
input_embeds_arr[0][2] = caption_embedding

# TODO: generate attn_mask_arr, input_embeds_arr

output_ids = t5.generate(attention_mask=attn_mask_arr, inputs_embeds=input_embeds_arr, max_length=768, num_beams=10)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

# Trying GPT-2

from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config

In [ ]:
GPT2Config.from_pretrained('gpt2')

In [ ]:
scene_caption = '1123 2 3 4 5 6 7'.split(" ")
import itertools
first_half_of_whisper_caption  = list(itertools.islice(scene_caption, 0, len(scene_caption)//2))
second_half_of_whisper_caption = list(itertools.islice(scene_caption, len(scene_caption)//2, None))
" ".join(first_half_of_whisper_caption)

In [ ]:
second_half_of_whisper_caption

In [ ]:
n = len(scene_caption)//2
parts = [scene_caption[i:i+n] for i in range(0, len(scene_caption), n)]
parts